In [73]:
from pathlib import Path
import pandas as pd
import json
import re
from collections import Counter

match_json_path = Path("/home/rjslater/Documents/Projects/UCR6-Stats-Bot/data/2_decoded_replays/Match-2024-04-14_17-41-21-37.json")

In [74]:
def get_players(match_json_path: Path):
    with open(match_json_path, "r") as f:
        match_json = json.load(f)

    players = []
    for round in match_json["rounds"]:
        teams = [team["name"] for team in round["teams"]]
        for player in round["players"]:
            team_name = teams[player["teamIndex"]] if teams[player["teamIndex"]] not in {"YOUR TEAM", "OPPONENTS"} else ""
            player_dict = {"id": player["profileID"], "name": player["username"], "team": team_name}
            # If player_dict[id] not in any dict in players, add this new player
            if any(player_dict["id"] == p["id"] for p in players):
                continue
            else:
                players.append(player_dict)

    return pd.DataFrame(players)

In [75]:
player_info_df = pd.DataFrame()
for match_json_path in Path("/home/rjslater/Documents/Projects/UCR6-Stats-Bot/data/2_decoded_replays/").glob("*.json"):
    player_info_df = pd.concat((player_info_df, get_players(match_json_path)), ignore_index=True)

player_info_df.drop_duplicates(subset="id", inplace=True)
player_info_df.sort_values(["team", "name"], inplace=True)
player_info_df.to_csv("/home/rjslater/Documents/Projects/UCR6-Stats-Bot/data/3_tables/player_info.csv", index=False)
player_info_df

,id,name,team
369,def89994-76a4-4eb2-810f-f8556fefcccd,fransce.UCD,
25,8b27aa51-fedc-42d2-bdcf-9f8f0bd8963e,Front.APP,App State
26,0b5d100d-9e54-4b2c-8a27-d5f6e5737451,Gluee.APP,App State
29,3654c484-5c63-4a64-b2b4-23fa94108da8,bam.APP,App State
28,6acf387d-4705-4681-bbf0-50eb1a2a21eb,ryansosa,App State
...,...,...,...
73,324d21e6-57a4-434f-b2fd-faa6de92cb8f,Buka.M80,UKY
71,16f2b53e-962f-4d6d-b489-7ff587cf38f6,Crypt.PSG,UKY
74,e33d7a41-cab9-4d93-b058-d3a4eea83b1b,Paladin.sQ,UKY
70,806373c2-59a2-45e4-b89b-f89cae5c6581,Tejatt1,UKY


In [76]:
def get_teams(match_json_path: Path):
    with open(match_json_path, "r") as f:
        match_json = json.load(f)

    teams = []
    round = match_json["rounds"][0]
    for team_index, team in enumerate(round["teams"]):
        team_name = team["name"] if team["name"] not in {"YOUR TEAM", "OPPONENTS"} else ""
        players = sorted([player["profileID"] for player in round["players"] if player["teamIndex"] == team_index])
        teams.append({"name": team_name, "players": players})

    return teams

In [77]:
def condense_teams(team_list, player_info_df):
    def are_same_team(players1, players2):
        """Check if two rosters have at least 3 players in common."""
        return len(set(players1) & set(players2)) >= 3

    def extract_tag(player_name):
        """Extract the tag from a player's username (e.g., `Player.TAG`)."""
        if "." in player_name:
            return player_name.split(".")[-1]
        return None

    def get_player_name(player_id):
        """Look up the player name from the DataFrame using the player ID."""
        result = player_info_df.loc[player_info_df["id"] == player_id, "name"]
        return result.iloc[0] if not result.empty else f"Unknown({player_id})"

    unique_teams = []

    for team in team_list:
        team_matched = False
        for unique_team in unique_teams:
            if are_same_team(team["players"], unique_team["players"]):
                # Merge players
                unique_team["players"] = list(set(unique_team["players"]) | set(team["players"]))
                # Track name usage
                unique_team["name_count"][team["name"]] += 1
                team_matched = True
                break

        if not team_matched:
            # Add a new unique team
            unique_teams.append({"name_count": Counter([team["name"]]), "players": team["players"]})

    # Finalize the unique teams with their most frequent names
    for unique_team in unique_teams:
        # Pick the most frequent non-empty name
        most_frequent_name = max((name for name in unique_team["name_count"] if name.strip()), key=lambda x: unique_team["name_count"][x], default="")

        # Guess team name if none is available
        if not most_frequent_name.strip():
            # Convert player IDs to names and count tags
            player_names = [get_player_name(player_id) for player_id in unique_team["players"]]
            player_tags = [extract_tag(player_name) for player_name in player_names if extract_tag(player_name)]
            tag_counts = Counter(player_tags)
            guessed_name = max(tag_counts, key=tag_counts.get, default="").strip()

            most_frequent_name = f"{guessed_name}" if guessed_name else "Unknown Team"

        unique_team["name"] = most_frequent_name
        del unique_team["name_count"]  # Clean up temporary counter

    return unique_teams

In [78]:
# # Example Input
# team_data = [
#     {"name": "Team Alpha", "players": [1, 2, 3, 4, 5]},
#     {"name": "Alpha Squad", "players": [1, 2, 6, 7, 8]},
#     {"name": "Team Beta", "players": [9, 10, 11, 12, 13]},
#     {"name": "Beta", "players": [9, 10, 14, 15, 16]},
#     {"name": "", "players": [1, 2, 3, 17, 18]},
# ]

# # Example Player Info DataFrame
# player_info_data = {
#     "id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
#     "name": [
#         "Alice.TAG",
#         "Bob.TAG",
#         "Charlie.TAG",
#         "David",
#         "Eve",
#         "Frank.TAG",
#         "George",
#         "Harry",
#         "Isaac.BETA",
#         "Jack.BETA",
#         "Kevin.BETA",
#         "Liam",
#         "Mike",
#         "Nina.BETA",
#         "Oscar",
#         "Paul",
#         "Tom.TAG",
#         "Jerry.TAG",
#     ],
# }
# player_info_df = pd.DataFrame(player_info_data)

# # Condense Teams
# unique_teams = condense_teams(team_data, player_info_df)

# # Output Results
# for team in unique_teams:
#     print(f"Name: {team['name']}, Players: {team['players']}")

In [79]:
team_instances = []
for match_json_path in Path("/home/rjslater/Documents/Projects/UCR6-Stats-Bot/data/2_decoded_replays").rglob("*.json"):
    team_instances.extend(get_teams(match_json_path))
for team in team_instances:
    print(team)
print()
unique_teams = condense_teams(team_instances, get_players(match_json_path))
for team in unique_teams:
    print(f"Name: {team['name']}, Roster: {team['players']}")

team_info_df = pd.DataFrame(unique_teams, columns=["name", "players"])
team_info_df.to_csv("/home/rjslater/Documents/Projects/UCR6-Stats-Bot/data/3_tables/teams.csv", index=False)

{'name': 'Cincinnati Red', 'players': ['2b6c9fd2-97b3-475f-91be-9fac07a81c94', '602ab4bc-8c6f-400a-a690-d0adc0cdbf58', 'c4a342d3-ea0d-4efe-93cf-7f59a56fef2b', 'df515609-2b3a-4053-ba21-3bc080b7254c', 'f2d9deea-6c23-4812-a052-eb3bbcc0e1d3']}
{'name': 'Kennesaw', 'players': ['7d2003f9-8945-496e-91dc-87decab9c552', '7e5c9b59-2745-4126-bca4-3394eadaa7de', 'dda2fdc1-071a-46fa-9e90-c64a0a96c4ea', 'e672fcda-c74d-41f6-a540-ed52783f9bca', 'f1256865-ad88-41a2-ba5f-18aeae19ff10']}
{'name': 'PSU Blue', 'players': ['068797b7-fc88-4247-8ead-bde6f25a7a88', '4460678a-b873-4bf7-ba14-5ed47bc1607f', '4af5baf5-5c7c-4551-9724-f3f75bfcdf83', 'baa5484a-5479-497e-95d7-28f333bc39ba', 'd54ee3ea-1622-41fe-b5d8-9b55f3480187']}
{'name': 'OSU Grey', 'players': ['124c8366-bba0-40fa-97e7-9e187acc2d82', '283d6c2e-d939-4bb7-9636-b1761dcd8e09', 'e6b0524a-67ad-44dc-a95c-f39735623c56', 'ebc723f4-4eca-47a2-8fbe-67fc14fc2679', 'ecaa3e99-b945-4ddb-aff6-1e4e7ed53c13']}
{'name': 'Cincinnati', 'players': ['2b6c9fd2-97b3-475f-91b

In [85]:
for _, row in team_info_df.iterrows():
    print(row["name"])
    for player_id in row["players"]:
        player_name = player_info_df.loc[player_info_df["id"] == player_id, "name"].iloc[0]
        print("\t" + player_name)

Cincinnati Red
	pupP
	K0Dl4K
	DuckDuk_
	Brij-
	Darkmatter.UC
Kennesaw
	BabyExploder.
	Plantgazer
	meinheiden
	Soteira
	Pawz-
PSU Blue
	YungGurv.PSU
	AtDabeach.PSU
	Whalecookie.PSU
	Kuhnzy.PSU
	Chimpingus.PSU
tOSU
	Mamba.
	n8-tOSU
	Rno.
	zjoc
	Toaster-tOSU
	Mustrd
App State
	bam.APP
	Gluee.APP
	ryansosa
	wong.APP
	Front.APP
UC Acad
	Naudu.UC
	greenbean.UC
	Popzao
	Cheeseboss.UC
	Morg.UC
	DLCV2
EKU
	Arcck.EKU
	Boxn.EKU
	Think.EKU
	Sinnn.EKU
	Lomo.EKU
MU
	Loochy.1k
	CheeseCurl.MU
	SMFBones.MU
	Zmart45
	Reserve.MU
UKY
	Buka.M80
	Paladin.sQ
	Crypt.PSG
	Tejatt1
	Bane.UKY
SCC
	koreyrob.SCC
	Rapid.SCC
	SaltyBoi.SCC
	Charm.SCC
	Jockz.SCC
Kent State
	Snookie.exe
	MrMeady
	IIsBestReader.
	Crusade.-
	praizy
	Strikkon
Fanshawe
	Masonlet.FUEL
	purzaa.FUEL
	Brendo.FUEL
	LondonR6
	Slip.FUEL
LVC
	MrDavisTV.LVC
	Plexios.LVC
	Empor1um.LVC
	TheSenate.LVC
	Reliable.LVC
DRX
	ItalianMafiaDRX
	Ducks.DRX
	Sirbuttalock
	Koake.
	osd.DRX
	Hades.DRX
UCD
	Guac.UCD
	Critt.UCD
	Laosley.UCD
	ilQwQli.UCD
	fransce.UCD
	